# Applicability of training ML model with TDA using Delaunay-Rips complex vs. using Rips vs. using Alpha
Author: Amish Mishra  
Date: November 1, 2022

## Notes
* We will use DR for "Delaunay-Rips"
* We will refer to the pipeline that uses DR, Rips, or Alpha for training/validating the corresponding ML model as the "DR method", "Rips method", or "Alpha method", respectively.

## Import the necessary libraries

## 1. Generate Persistence Statistics from Persistence Diagrams using DR, Rips, and Alpha

In [3]:
from persistence_stats import generate_training_validation_pers_stats
types = ['Training', 'Validation']
methods = ['rips', 'alpha', 'del_rips']
for t in types:
    for m in methods:
        generate_training_validation_pers_stats(t, m)

=========== Generating Training data persistence stats ===========
---- Using rips ----
CGMH_preprocessed_data/Training/1.csv
Loading CGMH_preprocessed_data/Training/1.csv


KeyboardInterrupt: 

## 2. Train ML models (SVM) based on Persistence Statistics

In [1]:
from ml_classifiers import generate_ml_classifiers
func_arr = ['rips', 'alpha', 'del_rips']
for func in func_arr:
    generate_ml_classifiers(func)

Training classifer based on rips


## 3. Validate ML models

## 4. Generate performance metrics